# Analyser des données depuis des API Web

Python permet de s'interfacer avec de nombreux services web (API). 

Dans cette partie nous verrons : 

- Ce qu'est une API 
- Comment Python et Pandas permettent de transformer des données brutes en un DataFrame exploitable

## Introduction aux API Web

Une API (Application Programming Interface) permet de récupérer des des données mises à disposition par des services Web accessibles depuis une adresse particulière. 

La plupart des services Web (Google Analytics, Mixpanel, Twitter, Facebook, le New York Times etc..) dispose d'API ce qui en fait une source de données de choix. 

Python et Pandas sont d'excellents outils pour récupérer, transformer et analyser ces données web. 

### URLs 

Une URL (Unique Ressource Locator) permet d'identifier de manière unique une ressource sur Internet. Prenons l'URL suivante : 

Cette URL se décompose en blocs logiques : 

- ***http://*** = le schéma, détermine le protocole que nous utilisons pour communiquer avec le service Web. Nous utiliserons principalement le protocole **http**
- ***www.example.com*** = l'hôte qui détermine sur quel serveur de l'internet nous allons communiquer
- ***/foo/bar*** = spécifie le chemin où se trouve la ressource
- ***?arg1=baz&arg2=quux*** les paramètres de la requête

### HTML vs JSON

- HTML permet de renvoyer de la donnée pour un utilisateur. Lorsqu'une requête est faite sur un serveur, celui-ci renvoie des éléments (html, css,js, etc..) permettant au navigateur d'afficher une page web **user-friendly**

<img src='images/html.png' width=500>

- JSON permet de renvoyer de la donnée structurée, exploitable par une machine. Lorsqu'un service doit communiquer avec un autre service, les données renvoyées sont **machine-friendly**. Plusieurs formats sont possibles (CSV, JSON, XML etc..)

<img src='images/json.png' width=500>

Le format JSON (Javascript Object Notation) est un des formats préférés des Web API. 

Il peut être composé de deux structures : 

- Dictionnaire clé / valeur
- Tableaux d'objets 

Voici un exemple de JSON : 

In [ ]:
{
     "firstName": "John",
     "lastName": "Smith",
     "address": {
         "streetAddress": "21 2nd Street",
         "city": "New York",
         "state": "NY",
         "postalCode": 10021
     },
     "phoneNumbers": [
         "212 555-1234",
         "646 555-4567"
     ]
 }

### Exemples d'API populaires

La plupart des services Web disposent d'API. Elles sont également documentées voici quelques exemples : 

- Facebook - https://developers.facebook.com/tools/explorer/
- Twitter - https://dev.twitter.com/rest/tools/console
- Google Analytics - https://ga-dev-tools.appspot.com/explorer/
- The New York Times - http://developer.nytimes.com/docs

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import requests

#### Api simple

In [73]:
#Adresse de l'api
api_astros = 'http://api.open-notify.org/astros.json'
#On utilise requests pour charger la page
r = requests.get(api_astros)
#On vérifie le si la requète s'est bien déroulée (200 = ok)
r.status_code

200

In [6]:
#On extrait les données json
result_api = r.json()

In [7]:
result_api

{'number': 13,
 'message': 'success',
 'people': [{'name': 'Mark Vande Hei', 'craft': 'ISS'},
  {'name': 'Pyotr Dubrov', 'craft': 'ISS'},
  {'name': 'Anton Shkaplerov', 'craft': 'ISS'},
  {'name': 'Zhai Zhigang', 'craft': 'Shenzhou 13'},
  {'name': 'Wang Yaping', 'craft': 'Shenzhou 13'},
  {'name': 'Ye Guangfu', 'craft': 'Shenzhou 13'},
  {'name': 'Raja Chari', 'craft': 'ISS'},
  {'name': 'Tom Marshburn', 'craft': 'ISS'},
  {'name': 'Kayla Barron', 'craft': 'ISS'},
  {'name': 'Matthias Maurer', 'craft': 'ISS'},
  {'name': 'Oleg Artemyev', 'craft': 'ISS'},
  {'name': 'Denis Matveev', 'craft': 'ISS'},
  {'name': 'Sergey Korsakov.', 'craft': 'ISS'}]}

**On cherche les clés**

In [8]:
result_api.keys()

dict_keys(['number', 'message', 'people'])

In [10]:
# On peut rentrer le tout dans une DataFrame
pd.DataFrame(result_api['people'])

,name,craft
0,Mark Vande Hei,ISS
1,Pyotr Dubrov,ISS
2,Anton Shkaplerov,ISS
3,Zhai Zhigang,Shenzhou 13
4,Wang Yaping,Shenzhou 13
5,Ye Guangfu,Shenzhou 13
6,Raja Chari,ISS
7,Tom Marshburn,ISS
8,Kayla Barron,ISS
9,Matthias Maurer,ISS


### Passer des paramètres

In [41]:
#On passe le paramètre de race labrador directement dans l'url
url = "https://api.thedogapi.com/v1/breeds/search?q=labrador"
r = requests.get(url)
r.json()

[{'weight': {'imperial': '55 - 80', 'metric': '25 - 36'},
  'height': {'imperial': '21.5 - 24.5', 'metric': '55 - 62'},
  'id': 149,
  'name': 'Labrador Retriever',
  'bred_for': 'Water retrieving',
  'breed_group': 'Sporting',
  'life_span': '10 - 13 years',
  'temperament': 'Kind, Outgoing, Agile, Gentle, Intelligent, Trusting, Even Tempered',
  'reference_image_id': 'B1uW7l5VX'}]

In [44]:
#Plus propre, on la passe dans les paramètres
param = {"q": "labrador"}
url = "https://api.thedogapi.com/v1/breeds/search"
r = requests.get(url, params=param)
r.json()

[{'weight': {'imperial': '55 - 80', 'metric': '25 - 36'},
  'height': {'imperial': '21.5 - 24.5', 'metric': '55 - 62'},
  'id': 149,
  'name': 'Labrador Retriever',
  'bred_for': 'Water retrieving',
  'breed_group': 'Sporting',
  'life_span': '10 - 13 years',
  'temperament': 'Kind, Outgoing, Agile, Gentle, Intelligent, Trusting, Even Tempered',
  'reference_image_id': 'B1uW7l5VX'}]

In [45]:
params = {"q": "poodle"}
url = "https://api.thedogapi.com/v1/breeds/search"
r = requests.get(url, params=params)
r.json()

[{'weight': {'imperial': '40 - 55', 'metric': '18 - 25'},
  'height': {'imperial': '16 - 24', 'metric': '41 - 61'},
  'id': 195,
  'name': 'Poodle',
  'bred_for': 'Retrieving through water',
  'breed_group': 'Non-Sporting',
  'life_span': '14 - 18 years',
  'temperament': 'Alert, Intelligent, Faithful, Active, Instinctual, Trainable'},
 {'weight': {'imperial': '15 - 17', 'metric': '7 - 8'},
  'height': {'imperial': '11 - 15', 'metric': '28 - 38'},
  'id': 196,
  'name': 'Poodle (Miniature)',
  'life_span': '12 – 15 years',
  'reference_image_id': 'Hkxk4ecVX'},
 {'weight': {'imperial': '6 - 9', 'metric': '3 - 4'},
  'height': {'imperial': '9 - 11', 'metric': '23 - 28'},
  'id': 197,
  'name': 'Poodle (Toy)',
  'life_span': '18 years',
  'reference_image_id': 'rJFJVxc4m'},
 {'weight': {'imperial': '5 - 10', 'metric': '2 - 5'},
  'height': {'imperial': '10', 'metric': '25'},
  'id': 249,
  'name': 'Toy Poodle',
  'breed_group': 'Toy',
  'life_span': '14 - 18 years'}]

#### Utiliser des acces key

In [57]:
url = "https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos"
# Replacer DEMO_KEY si vous en avez généré une
# générer une clé api : https://api.nasa.gov:443
# api_key = "MkCWZJ6NMlICFvDIXEMEGrBtOgbvV4NUInTgUOx9"
#On cherche les photos de mars prises le 1er mars 2022
api_key = "DEMO_KEY"
params = {"api_key": api_key, "earth_date": "2022-03-01"}
response = requests.get(url, params=params)
response

<Response [200]>

In [59]:
json_data = response.json()

In [60]:
json_data

{'photos': [{'id': 943583,
   'sol': 3401,
   'camera': {'id': 20,
    'name': 'FHAZ',
    'rover_id': 5,
    'full_name': 'Front Hazard Avoidance Camera'},
   'img_src': 'https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/03401/opgs/edr/fcam/FLB_699416473EDR_F0932578FHAZ00302M_.JPG',
   'earth_date': '2022-03-01',
   'rover': {'id': 5,
    'name': 'Curiosity',
    'landing_date': '2012-08-06',
    'launch_date': '2011-11-26',
    'status': 'active'}},
  {'id': 943584,
   'sol': 3401,
   'camera': {'id': 20,
    'name': 'FHAZ',
    'rover_id': 5,
    'full_name': 'Front Hazard Avoidance Camera'},
   'img_src': 'https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/03401/opgs/edr/fcam/FRB_699416473EDR_F0932578FHAZ00302M_.JPG',
   'earth_date': '2022-03-01',
   'rover': {'id': 5,
    'name': 'Curiosity',
    'landing_date': '2012-08-06',
    'launch_date': '2011-11-26',
    'status': 'active'}},
  {'id': 943585,
   'sol': 3401,
   'camera': {'id': 21,
   

In [63]:
json_data.keys()

dict_keys(['photos'])

In [64]:
list_photos = json_data['photos']

In [67]:
# Nombre de photos
len(list_photos)

118

In [72]:
list_photos[0]["img_src"]

'https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/03401/opgs/edr/fcam/FLB_699416473EDR_F0932578FHAZ00302M_.JPG'

#### Exemples avec l'api de Wikipedia

### Chercher le début d'un article sur le langage Python

In [116]:
subject = 'Python (langage)'
url = 'https://fr.wikipedia.org/w/api.php'
params = {
        'action': 'query',
        'format': 'json',
        'titles': subject,
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }
 
response = requests.get(url, params=params)
data = response.json()

In [117]:
data

{'batchcomplete': '',
 'query': {'pages': {'2340': {'pageid': 2340,
    'ns': 0,
    'title': 'Python (langage)',
    'extract': "Python (prononcé /pi.tɔ̃/) est un langage de programmation interprété, multi-paradigme et multiplateformes. Il favorise la programmation impérative structurée, fonctionnelle et orientée objet. Il est doté d'un typage dynamique fort, d'une gestion automatique de la mémoire par ramasse-miettes et d'un système de gestion d'exceptions ; il est ainsi similaire à Perl, Ruby, Scheme, Smalltalk et Tcl.\nLe langage Python est placé sous une licence libre proche de la licence BSD et fonctionne sur la plupart des plates-formes informatiques, des smartphones aux ordinateurs centraux, de Windows à Unix avec notamment GNU/Linux en passant par macOS, ou encore Android, iOS, et peut aussi être traduit en Java ou .NET. Il est conçu pour optimiser la productivité des programmeurs en offrant des outils de haut niveau et une syntaxe simple à utiliser.\nIl est également apprécié

In [118]:
pages = data['query']['pages']

In [123]:
pages[list(pages.keys())[0]]['extract'][:192]

'Python (prononcé /pi.tɔ̃/) est un langage de programmation interprété, multi-paradigme et multiplateformes. Il favorise la programmation impérative structurée, fonctionnelle et orientée objet.'

### Chercher les article de Wikipedia France qui ont "Python" dans leur titre

In [124]:
query = 'python'
 
url = 'https://fr.wikipedia.org/w/api.php'
params = {
            'action':'query',
            'format':'json',
            'list':'search',
            'utf8':1,
            'srsearch':query
        }
 
data = requests.get(url, params=params).json()
 
for i in data['query']['search']:
    print(i['title'], ' - Word count: ', i['wordcount'])

Python (langage)  - Word count:  7484
Monty Python  - Word count:  8936
Python  - Word count:  232
Monty Python : Sacré Graal !  - Word count:  3334
PYTHON  - Word count:  641
Python réticulé  - Word count:  1486
Python royal  - Word count:  1261
Python (mythologie)  - Word count:  500
Python (serpent)  - Word count:  220
Liste de logiciels Python  - Word count:  1559


#### Exercices
#### Recherche de coordonnées géographiques d'une adresse  dans la base adresse nationale

**Documentation**
- https://github.com/BaseAdresseNationale/api-fantoir/blob/master/README.md#api
- https://adresse.data.gouv.fr/api-doc/adresse

**url de base :**  
https://api-adresse.data.gouv.fr/search/  
Trouver les coordonnées géographique du "8 avenue de la République à Paris"  
En utilisant request, get et au moins un paramètre

**Faire la liste des et Avenues de la commune de Vincennes (dans une DataFrame)**  
Attention, le code commune n'est pas la même chose que le code postal

Documentation  
https://github.com/BaseAdresseNationale/api-fantoir/blob/master/README.md#api